# Multi-Agent Support
This is an example implementation of tracking events from two separate agents

In [1]:
import agentops
from agentops.agent import track_agent
from dotenv import load_dotenv
import os
import openai
import logging
import traceback

from IPython.display import display, Markdown

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', "<your_openai_key>")
AGENTOPS_API_KEY = os.getenv('AGENTOPS_API_KEY', "<your_agentops_key>")
logging.basicConfig(level=logging.INFO)

In [3]:
ao_client = agentops.Client(AGENTOPS_API_KEY, endpoint='http://localhost:8000')
openai_client = openai.Client()

INFO:root:View info on this session at https://agentops.ai/dashboard?session_id=5193a9a7-aa7f-485d-843b-36518566e983


Now lets create a few agents!

In [12]:
res = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": "You are not a tracked agent"},
    {"role": "user", "content": "Say hello"}]
)
# traceback.print_exc()
# raise Exception()
res.choices[0].message.content

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Unable to parse a chunk for LLM call {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': 'You are not a tracked agent'}, {'role': 'user', 'content': 'Say hello'}]} - skipping upload to AgentOps


'Hello! How can I assist you today?'

In [5]:
@track_agent(name='qa')
class QaAgent:
    def completion(self, prompt: str):
        res = openai_client.chat.completions.create(model='gpt-3.5-turbo', messages=[{"role": "system", "content": "You are a qa engineer and only output python code, no markdown tags."},
    {"role": "user", "content": prompt}], temperature=0.5)
        return res.choices[0].message.content
        
@track_agent(name='engineer')
class EngineerAgent:
    def completion(self, prompt: str):
        res = openai_client.chat.completions.create(model='gpt-3.5-turbo', messages=[{"role": "system", "content": "You are a software engineer and only output python code, no markdown tags."},
    {"role": "user", "content": prompt}], temperature=0.5)
        return res.choices[0].message.content

In [6]:
qa = QaAgent()
qa2 = QaAgent()
engineer = EngineerAgent()

In [7]:
print(engineer._ao_agent_name)
print(qa._ao_agent_id)
print(qa2._ao_agent_id)

engineer
fc947d29-c909-4470-bd33-f0da4687f81a
3ecfe8d2-df1a-4189-85b1-0736d4314843


In [8]:
generated_func = engineer.completion("Write a python function that accepts two numbers and multiplies them together, then divides by two. No example.")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:LLM call from agent named: engineer


Unable to parse a chunk for LLM call {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': 'You are a software engineer and only output python code, no markdown tags.'}, {'role': 'user', 'content': 'Write a python function that accepts two numbers and multiplies them together, then divides by two. No example.'}], 'temperature': 0.5} - skipping upload to AgentOps


In [9]:
display(Markdown('```python\n' + generated_func + '\n```'))


```python
def multiply_and_divide(num1, num2):
    result = (num1 * num2) / 2
    return result
```

In [10]:
generated_test = qa.completion("Write a python unit test that test the following function: \n " + generated_func)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:LLM call from agent named: qa


Unable to parse a chunk for LLM call {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': 'You are a qa engineer and only output python code, no markdown tags.'}, {'role': 'user', 'content': 'Write a python unit test that test the following function: \n def multiply_and_divide(num1, num2):\n    result = (num1 * num2) / 2\n    return result'}], 'temperature': 0.5} - skipping upload to AgentOps


In [11]:
display(Markdown('```python\n' + generated_test + '\n```'))

```python
import unittest

def multiply_and_divide(num1, num2):
    result = (num1 * num2) / 2
    return result

class TestMultiplyAndDivide(unittest.TestCase):

    def test_multiply_and_divide(self):
        self.assertEqual(multiply_and_divide(4, 2), 4.0)
        self.assertEqual(multiply_and_divide(5, 3), 7.5)
        self.assertEqual(multiply_and_divide(0, 10), 0.0)
        self.assertEqual(multiply_and_divide(-6, 3), -9.0)

if __name__ == '__main__':
    unittest.main()
```